# Aggregating Axiom Preferences for Learning to Rank

The notebook below exemplifies how `ir_axioms` can be used to generate features for arbitrary learning-to-rank approaches like LambdMART in PyTerrier by aggregating pairwise preferences from different axioms.
We use run files and qrels from the passage retrieval task of the TREC Deep Learning track in 2019 and 2020 as example (using BM25 as a baseline).
In this notebook, we first generate learning-to-rank features by aggregating preferences from 2019 qrels and topics. Then we train a LambdaMART re-ranker with the generated axiomatic features, re-rank using the trained LambdaMART re-ranker, and evaluate nDCG@10, reciprocal rank, and average precision for the baseline and the re-ranked pipeline using PyTerrier's standard `Experiment` functionality.


## Preparation

Install the `ir_axioms` framework and [PyTerrier](https://github.com/terrier-org/pyterrier). In Google Colab, we do this automatically.


In [ ]:
from sys import modules

if "google.colab" in modules:
    !pip install -q ir_axioms[experiments] python-terrier

We initialize PyTerrier and import all required libraries and load the data from [ir_datasets](https://ir-datasets.com/).


In [ ]:
from pyterrier import started, init

if not started():
    init(tqdm="auto")

## Datasets and Index

Using PyTerrier's `get_dataset()`, we load the MS MARCO passage ranking dataset.


In [ ]:
from pyterrier.datasets import get_dataset, Dataset

# Load dataset.
dataset_name = "msmarco-passage"
dataset: Dataset = get_dataset(f"irds:{dataset_name}")
dataset_train: Dataset = get_dataset(f"irds:{dataset_name}/trec-dl-2019/judged")
dataset_test: Dataset = get_dataset(f"irds:{dataset_name}/trec-dl-2020/judged")

Now define paths where we will store temporary files, datasets, and the search index.


In [ ]:
from pathlib import Path

cache_dir = Path("cache/")
index_dir = cache_dir / "indices" / dataset_name.split("/")[0]

If the index is not ready yet, now is a good time to create it and index the MS MARCO passages.
(Lean back and relax as this may take a while...)


In [ ]:
from pyterrier.index import IterDictIndexer

if not index_dir.exists():
    indexer = IterDictIndexer(str(index_dir.absolute()))
    indexer.index(dataset.get_corpus_iter(), fields=["text"])

## Baseline Run

We use PyTerrier's `BatchRetrieve` to create a baseline search pipeline for retrieving with BM25 from the index we just created.


In [ ]:
from pyterrier.batchretrieve import BatchRetrieve

bm25 = BatchRetrieve(str(index_dir.absolute()), wmodel="BM25")

## Import Axioms

Here we're listing which axioms we want to use in our experiments.
Because some axioms require API calls or are computationally expensive, we cache all axioms using `ir_axiom`'s tilde operator (`~`).


In [ ]:
from ir_axioms.axiom import (
    ArgUC,
    QTArg,
    QTPArg,
    aSL,
    PROX1,
    PROX2,
    PROX3,
    PROX4,
    PROX5,
    TFC1,
    TFC3,
    RS_TF,
    RS_TF_IDF,
    RS_BM25,
    RS_PL2,
    RS_QL,
    AND,
    LEN_AND,
    M_AND,
    LEN_M_AND,
    DIV,
    LEN_DIV,
    M_TDC,
    LEN_M_TDC,
    STMC1,
    STMC1_f,
    STMC2,
    STMC2_f,
    LNC1,
    TF_LNC,
    LB1,
    REG,
    ANTI_REG,
    REG_f,
    ANTI_REG_f,
    ASPECT_REG,
    ASPECT_REG_f,
    ORIG,
)

axioms = [
    ~ArgUC(),
    ~QTArg(),
    ~QTPArg(),
    ~aSL(),
    ~LNC1(),
    ~TF_LNC(),
    ~LB1(),
    ~PROX1(),
    ~PROX2(),
    ~PROX3(),
    ~PROX4(),
    ~PROX5(),
    ~REG(),
    ~REG_f(),
    ~ANTI_REG(),
    ~ANTI_REG_f(),
    ~ASPECT_REG(),
    ~ASPECT_REG_f(),
    ~AND(),
    ~LEN_AND(),
    ~M_AND(),
    ~LEN_M_AND(),
    ~DIV(),
    ~LEN_DIV(),
    ~RS_TF(),
    ~RS_TF_IDF(),
    ~RS_BM25(),
    ~RS_PL2(),
    ~RS_QL(),
    ~TFC1(),
    ~TFC3(),
    ~M_TDC(),
    ~LEN_M_TDC(),
    ~STMC1(),
    ~STMC1_f(),
    ~STMC2(),
    ~STMC2_f(),
    ORIG(),
]

To reduce the preference matrices to lists of three features, we define three aggregation functions.
In this exemplary aggregations, we just count how often an axiom wants to change the original ordering in either direction.


In [ ]:
aggregations = [
    lambda prefs: sum(p >= 0 for p in prefs) / len(prefs),
    lambda prefs: sum(p == 0 for p in prefs) / len(prefs),
    lambda prefs: sum(p <= 0 for p in prefs) / len(prefs),
]

## Aggregating Axiomatic Features

With the axioms and aggregation functions, we create axiomatic preference features for the top-20 results from the baseline ranking.


In [ ]:
from ir_axioms.backend.pyterrier.transformers import AggregatedAxiomaticPreferences

features = bm25 % 20 >> AggregatedAxiomaticPreferences(
    axioms=axioms,
    index=index_dir,
    aggregations=aggregations,
    dataset=dataset_name,
    cache_dir=cache_dir,
    verbose=True,
)

This example shows how the features look like for the first topic of the training dataset.


In [ ]:
features.transform(dataset_train.get_topics()[:1])["features"]

## Learning to Rank

After aggregating the preference features, we can use the features with any learning-to-rank approach.
In this case, we initialize a LambdaMART ranker for optimizing nDCG@10 and apply it to our PyTerrier pipeline.


In [ ]:
from lightgbm import LGBMRanker
from pyterrier.ltr import apply_learned_model

lambda_mart = LGBMRanker(
    num_iterations=1000,
    metric="ndcg",
    eval_at=[10],
    importance_type="gain",
)
ltr = features >> apply_learned_model(lambda_mart, form="ltr")

We train the LamdaMART ranker with the training dataset (using the last 5 topics as the validation dataset).


In [ ]:
ltr.fit(
    dataset_train.get_topics()[:-5],
    dataset_train.get_qrels(),
    dataset_train.get_topics()[-5:],
    dataset_train.get_qrels(),
)

## Experimental Evaluation

Because our axiomatic re-rankers are PyTerrier modules, we can now use PyTerrier's `Experiment` interface to evaluate various metrics and to compare our new approach to the BM25 baseline ranking.
Refer to the PyTerrier [documentation](https://pyterrier.readthedocs.io/en/latest/experiments.html) to learn more about running experiments.
(We concatenate results from the Baseline ranking for the ranking positions after the top-20 using the `^` operator.)


In [ ]:
from pyterrier.pipelines import Experiment
from ir_measures import nDCG, MAP, RR

experiment = Experiment(
    [bm25, ltr ^ bm25],
    dataset_test.get_topics(),
    dataset_test.get_qrels(),
    [nDCG @ 10, RR, MAP],
    ["BM25", "Axiomatic LTR"],
    verbose=True,
)
experiment.sort_values(by="nDCG@10", ascending=False, inplace=True)

In [ ]:
experiment

## Extra: Feature Importances

Inspecting the feature importances from LambdaMART can help to identify axioms or aggregations that are not used for re-ranking.
If an axiom's feature importance is zero for most of your applications, you may consider omitting it from the ranking pipeline.


In [ ]:
from numpy import ndarray

feature_importance: ndarray = lambda_mart.feature_importances_.reshape(
    -1, len(aggregations)
)
feature_importance

In [ ]:
feature_importance.sum(0)

In [ ]:
feature_importance.sum(1)